In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')

import pyspark
from pyspark.sql.functions import lit, col, instr, expr, pow, round, bround, corr, count, mean, stddev_pop, min, max
from pyspark.sql.functions import monotonically_increasing_id, initcap, lower, upper, ltrim, rtrim, rpad, lpad, trim
from pyspark.sql.functions import regexp_replace, translate, regexp_extract, current_date, current_timestamp, struct
from pyspark.sql.functions import date_add, date_sub, datediff, months_between, to_date, to_timestamp, coalesce, split, size
from pyspark.sql.functions import array_contains, explode, udf
from pyspark.sql import HiveContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, when

from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType, FloatType, LongType

In [2]:
def get_Spark():

    conf = pyspark.SparkConf().setAll([
        ('spark.submit.deployMode', 'client'), # deploy in yarn-client or yarn-cluster
        ('spark.executor.memory', '8g'),       # memory allocated for each executor
        ('spark.executor.cores', '3'),         # number of cores for each executor
        ('spark.executor.instances', '10'),    # number of executors in total
        ('spark.yarn.am.memory', '24g')])      # memory for spark driver (application master)
    spark = SparkSession.builder \
    .master("yarn") \
    .appName("test") \
    .enableHiveSupport() \
    .config(conf = conf) \
    .getOrCreate()

    return spark

spark = get_Spark()
spark_context = spark.sparkContext
hc = HiveContext(spark_context)

In [ ]:
"""
import datetime
start_time = datetime.datetime.combine(d1, datetime.time(0, 0))
print (start_time)

def hive2pandas(hc, query):
    spark_df = hc.sql("""{}""".format(query))
    # Convert to pandas dataframe
    df = spark_df.toPandas()
    return df
"""

In [ ]:
"""
def addLocation(hc, p):
    #query = """SELECT distinct ROUND(CAST(start_loc_lat as float), 3) as loc_lat, ROUND(CAST(start_loc_lon as float), 3) as loc_lon from ubi.conv_trips_complete union SELECT distinct ROUND(CAST(end_loc_lat as float), 3) as loc_lat, ROUND(CAST(end_loc_lon as float), 3) as loc_lon from ubi.conv_trips_complete where start_day>='{0}' and start_day<='{1}'""".format(d1, d2)
    query = """SELECT distinct CAST(start_loc_lat as float) as loc_lat, CAST(start_loc_lon as float) as loc_lon from ubi.conv_trips_complete union SELECT distinct CAST(end_loc_lat as float) as loc_lat, CAST(end_loc_lon as float) as loc_lon from ubi.conv_trips_complete where start_day>='{0}' and start_day<='{1}'""".format(d1, d2)
    print(query)
    df=hive2pandas(hc, query)
    with open('/home/wchen/ubi2/latlon2location_' + str(p) + '.txt', 'w') as outfile:
        for x in range(len(df)):
            if(int(df.iloc[x,0] * 10) % 10 == p):
                g = geocoder.gaode([df.iloc[x,0], df.iloc[x,1]], method='reverse', key='27522a3d9da8f4e80d6580c80d010d4c')
                adds = g.address
                print(adds)
                if g.address is None:
                    adds=''
                elif type(g.address)==list:
                    adds=str(adds)
                outfile.write('%.3f'%(df.iloc[x,0]) + '\t' + '%.3f'%(df.iloc[x,1]) + '\t' + adds.encode('utf-8') + '\n')
    print('Done with location: ' + str(p))
    
    addLocation(hc,0)
"""

In [ ]:
infile='/home/wchen/dsc/file/combined.csv'

In [ ]:
import os
os.path.isfile(infile)

In [ ]:
import pandas as pd
df = pd.read_csv(infile, sep=',',  dtype={'loc_latitude': float, 'loc_longtitude':float, 'address':str}, encoding="utf-8")

In [ ]:
import geocoder

for x in range(len(df)):
    if pd.isna(df.iloc[x,2]):
        #print df.iloc[x]
        g = geocoder.gaode([df.iloc[x,0], df.iloc[x,1]], method='reverse', key='27522a3d9da8f4e80d6580c80d010d4c')
        adds = g.address
        if g.address is None:
            adds=''
        elif type(g.address)==list:
            adds=str(adds)
        df.iloc[x,2] = adds.encode('utf-8')
        #print df.iloc[x]

In [ ]:
df.head()

In [ ]:
import geocoder

for x in range(len(df)):
    if pd.isna(df.iloc[x,2]):
        print df.iloc[x]
print("Check completed.")

In [8]:
mySchema = StructType([StructField("loc_latitude", DoubleType(), True)\
               ,StructField("loc_longtitude", DoubleType(), True)\
               ,StructField("address", StringType(), True)])


In [ ]:
type(df['address'][0])

In [ ]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")
pd.to_numeric(df['loc_latitude'])
pd.to_numeric(df['loc_longtitude'])
df['address'] = str(df['address']).encode('utf-8')

In [5]:
#df.to_csv("df_temp.csv.gz", index=False, compression="gzip")
import pandas as pd
df_2 = pd.read_csv("df_temp.csv.gz", encoding="utf-8")
df_2.head()

,loc_latitude,loc_longtitude,address
0,23.089,113.280,广东省广州市海珠区昌岗街道细岗路106号晓园北小区
1,31.011,121.526,上海市闵行区浦江镇丰南路1270号
2,25.040,113.354,广东省韶关市乐昌市长来镇G4W3乐广高速
3,23.009,113.318,广东省广州市番禺区大石街道礼村中路56号主人花园
4,38.065,114.203,河北省石家庄市井陉县上安镇李志刚诊所


In [6]:
df=df_2

In [9]:
my_cols=["loc_latitude","loc_longtitude","address"]
#spark_df = hc.createDataFrame([tuple(x for x in record.tolist()) for record in df.to_records(index=False)], df.columns.tolist())
spark_df = hc.createDataFrame(df[my_cols], mySchema)

In [10]:
cols = [when(~col(x).isin("NULL", "NA", "NaN",""), col(x)).alias(x) for x in spark_df.columns]
spark_df = spark_df.select(*cols)
spark_df.registerTempTable('update_dataframe')
sql_cmd = """INSERT INTO TABLE ubi.address SELECT loc_latitude, loc_longtitude, address FROM update_dataframe"""
print(sql_cmd)
hc.sql(sql_cmd)
print('Table address creation done.')

INSERT INTO TABLE ubi.address SELECT loc_latitude, loc_longtitude, address FROM update_dataframe
Table address creation done.
